## Data preprocessing

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd /content/drive/MyDrive/COSRMAL_CHALLENGE/CORSMAL-Challenge-2022-Squids


/content/drive/MyDrive/COSRMAL_CHALLENGE/CORSMAL-Challenge-2022-Squids


In [3]:
import scipy
import librosa
import pandas as pd
import os
import numpy as np
from tqdm.notebook import tqdm
import cv2
import scipy.io.wavfile
import time
import IPython
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import Subset
import json
from PIL import Image
from torchvision import datasets, transforms
from dataset import MiniDataset
from loss import computeScoreType1, myLoss
from models import Net, effnetv2_xl, MobileNetV3_Large, mbv2_ca
from helper import train_image, evaluate_image

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [4]:
!cp /content/drive/MyDrive/COSRMAL_CHALLENGE/RGBD_new/crops_rgb_train.zip /content/crops_rgb_train.zip
!cp /content/drive/MyDrive/COSRMAL_CHALLENGE/RGBD_new/crops_rgb_test.zip /content/crops_rgb_test.zip
!cp /content/drive/MyDrive/COSRMAL_CHALLENGE/RGBD_new/crops_depth_train.zip /content/crops_depth_train.zip
!cp /content/drive/MyDrive/COSRMAL_CHALLENGE/RGBD_new/crops_depth_test.zip /content/crops_depth_test.zip
!cp /content/drive/MyDrive/COSRMAL_CHALLENGE/RGBD_new/labals_test.zip /content/labels_test.zip
!cp /content/drive/MyDrive/COSRMAL_CHALLENGE/RGBD_new/labals_train.zip /content/labels_train.zip


In [5]:
!unzip /content/crops_rgb_train.zip -d /content/
!unzip /content/crops_rgb_test.zip -d /content/
!unzip /content/crops_depth_train.zip -d /content/
!unzip /content/crops_depth_test.zip -d /content/
!unzip /content/labels_train.zip -d /content/
!unzip /content/labels_test.zip -d /content/

Archive:  /content/crops_rgb_train.zip
replace /content/train/crops_rgb/002215.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: Archive:  /content/crops_rgb_test.zip
replace /content/test/crops_rgb/000408.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: Archive:  /content/crops_depth_train.zip
replace /content/train/crops_depth/002215.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: Archive:  /content/crops_depth_test.zip
replace /content/test/crops_depth/000408.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: Archive:  /content/labels_train.zip
replace /content/train/labels/001861.npy? [y]es, [n]o, [A]ll, [N]one, [r]ename: Archive:  /content/labels_test.zip
replace /content/test/labels/000478.npy? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
os.makedirs('/content/train',exist_ok=True)
os.makedirs('/content/test',exist_ok=True)
!unzip /content/crops_rgb_train.zip -d /content/train/
!unzip /content/crops_rgb_test.zip -d /content/test/
!unzip /content/crops_depth_train.zip -d /content/train/
!unzip /content/crops_depth_test.zip -d /content/test/
!unzip /content/labels_train.zip -d /content/train/
!unzip /content/labels_test.zip -d /content/test

Archive:  /content/crops_rgb_train.zip
replace /content/train/train/crops_rgb/002215.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: Archive:  /content/crops_rgb_test.zip
replace /content/test/test/crops_rgb/000408.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: Archive:  /content/crops_depth_train.zip
replace /content/train/train/crops_depth/002215.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: Archive:  /content/crops_depth_test.zip
replace /content/test/test/crops_depth/000408.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

# Train

## Capacity

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

my_save_path = '/content/drive/MyDrive/COSRMAL_CHALLENGE'
bs = 2
train_split = 0.8
lr = 1e-4
epochs = 50
train_set = MiniDataset('/content/train/',
                        '/content/train/labels', 
                        '/content/train/crops_depth',
                        '/content/train/crops_rgb')
val_set = MiniDataset('/content/test/',
                        '/content/test/labels', 
                        '/content/test/crops_depth',
                        '/content/test/crops_rgb')
model = effnetv2_xl(num_classes=1).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr,  weight_decay=1e-5)

best_loss = float('inf')
best_acc = 0

num_train = len(train_set)
num_val = len(val_set)



train_loader   = DataLoader(train_set,
                            batch_size=bs,
                            shuffle=True,
                            num_workers=1)
val_loader   = DataLoader(val_set,
                          batch_size=bs,
                          shuffle=True,
                          num_workers=1)

for epoch in range(epochs):
  #start_time = time.time()
  
  loss_train, correct_train = train_image(model, train_loader, optimizer, device, criterion = myLoss)
  loss_val, correct_val = evaluate_image(model, val_loader, device, myLoss)
  #elapsed_time = time.time() - start_time
  print("{}/{} train loss:{:.4f} train acc:{:.2f}% val loss:{:.4f} val acc:{:.2f}%".format(
      epoch+1,epochs, loss_train / num_train, 100 * correct_train/num_train,
      loss_val /num_val, 100 * correct_val/num_val))
  

  # if loss_val < best_loss:
  #   best_loss = loss_val
  #   torch.save(model, os.path.join(base_path, 'audios', "bl-efficient-xl.pth"))
  
  if correct_val > best_acc:
    best_acc = correct_val
    torch.save(model.state_dict(), os.path.join(my_save_path, 
                                              'audios', 
                                              'RGBD',
                                              "XL-my{:.2f}.pth".format(100 * correct_val/num_val)))



Using device: cuda
1/50 train loss:0.5427 train acc:31.11% val loss:0.5720 val acc:26.78%
2/50 train loss:0.4562 train acc:48.15% val loss:0.4846 val acc:38.49%
3/50 train loss:0.3094 train acc:69.06% val loss:0.3300 val acc:67.00%
4/50 train loss:0.2892 train acc:71.08% val loss:0.3078 val acc:69.22%
5/50 train loss:0.2902 train acc:70.98% val loss:0.3136 val acc:68.64%
6/50 train loss:0.2866 train acc:71.34% val loss:0.3190 val acc:68.10%
7/50 train loss:0.2842 train acc:71.58% val loss:0.3055 val acc:69.45%
8/50 train loss:0.2883 train acc:71.17% val loss:0.3106 val acc:68.94%
9/50 train loss:0.2878 train acc:71.22% val loss:0.3123 val acc:68.77%
10/50 train loss:0.2875 train acc:71.25% val loss:0.3227 val acc:67.73%
11/50 train loss:0.2860 train acc:71.40% val loss:0.3280 val acc:67.20%
12/50 train loss:0.2822 train acc:71.78% val loss:0.3241 val acc:67.59%
13/50 train loss:0.2841 train acc:71.59% val loss:0.3093 val acc:69.07%
14/50 train loss:0.2835 train acc:71.65% val loss:0.31

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

my_save_path = '/content/drive/MyDrive/COSRMAL_CHALLENGE'
bs = 30
train_split = 0.8
lr = 1e-3
epochs = 100
train_set = MiniDataset('/content/train/',
                        '/content/train/labels', 
                        '/content/train/crops_depth',
                        '/content/train/crops_rgb')
val_set = MiniDataset('/content/test/',
                        '/content/test/labels', 
                        '/content/test/crops_depth',
                        '/content/test/crops_rgb')
model = MobileNetV3_Large(num_classes=1).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr,  weight_decay=1e-5)

best_loss = float('inf')
best_acc = 0

num_train = len(train_set)
num_val = len(val_set)





train_loader   = DataLoader(train_set,
                            batch_size=bs,
                            shuffle=True,
                            num_workers=1,
                            drop_last=True)
val_loader   = DataLoader(val_set,
                          batch_size=bs,
                          shuffle=True,
                          num_workers=1)

for epoch in range(epochs):
  #start_time = time.time()
  loss_train, correct_train = train_image(model, train_loader, optimizer, device, criterion = myLoss)
  loss_val, correct_val = evaluate_image(model, val_loader, device, myLoss)
  #elapsed_time = time.time() - start_time
  print("{}/{} train loss:{:.4f} train acc:{:.2f}% val loss:{:.4f} val acc:{:.2f}%".format(
      epoch+1,epochs, loss_train/num_train, 100 * correct_train/num_train,
      loss_val/num_val, 100 * correct_val/num_val))
  

  # if loss_val < best_loss:
  #   best_loss = loss_val
  #   torch.save(model, os.path.join(base_path, 'audios', "bl-efficient-xl.pth"))
  
  if correct_val > best_acc:
    best_acc = correct_val
    torch.save(model.state_dict(), os.path.join(my_save_path, 
                                              'audios', 
                                              'RGBD',
                                              "mobile{:.2f}.pth".format(100 * correct_val/num_val)))



Using device: cuda
1/100 train loss:0.5963 train acc:24.71% val loss:0.5315 val acc:35.67%
2/100 train loss:0.4706 train acc:38.27% val loss:0.7242 val acc:0.60%
3/100 train loss:0.3704 train acc:55.41% val loss:0.3444 val acc:60.08%
4/100 train loss:0.3108 train acc:65.45% val loss:0.4562 val acc:53.52%
5/100 train loss:0.2637 train acc:72.20% val loss:0.2807 val acc:71.67%
6/100 train loss:0.2341 train acc:75.72% val loss:0.4366 val acc:55.67%
7/100 train loss:0.2190 train acc:77.07% val loss:0.2541 val acc:74.59%
8/100 train loss:0.1995 train acc:79.21% val loss:0.2256 val acc:77.44%
9/100 train loss:0.1982 train acc:79.42% val loss:0.3874 val acc:61.26%
10/100 train loss:0.2175 train acc:77.48% val loss:0.5762 val acc:41.36%
11/100 train loss:0.2191 train acc:77.38% val loss:0.2387 val acc:76.13%
12/100 train loss:0.2662 train acc:72.18% val loss:0.2828 val acc:71.19%
13/100 train loss:0.2357 train acc:75.66% val loss:0.5781 val acc:41.99%
14/100 train loss:0.2013 train acc:79.18% 

## Mass estimation

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

my_save_path = '/content/drive/MyDrive/COSRMAL_CHALLENGE'
bs = 30
train_split = 0.8
lr = 1e-4
epochs = 100
train_set = MiniDataset('/content/train/',
                        '/content/train/labels', 
                        '/content/train/crops_depth',
                        '/content/train/crops_rgb',
                        ['container mass']
                        )
val_set = MiniDataset('/content/test/',
                      '/content/test/labels', 
                      '/content/test/crops_depth',
                      '/content/test/crops_rgb',
                      ['container mass']
                       
                      )
model = MobileNetV3_Large(num_classes=1).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr,  weight_decay=1e-5)

best_loss = float('inf')
best_acc = 0

num_train = len(train_set)
num_val = len(val_set)





train_loader   = DataLoader(train_set,
                            batch_size=bs,
                            shuffle=True,
                            num_workers=1,
                            drop_last=True)
val_loader   = DataLoader(val_set,
                          batch_size=bs,
                          shuffle=True,
                          num_workers=1)

for epoch in range(epochs):
  #start_time = time.time()
  loss_train, correct_train = train_image(model, train_loader, optimizer, device, criterion = myLoss)
  loss_val, correct_val = evaluate_image(model, val_loader, device, myLoss)
  #elapsed_time = time.time() - start_time
  print("{}/{} train loss:{:.4f} train acc:{:.2f}% val loss:{:.4f} val acc:{:.2f}%".format(
      epoch+1,epochs, loss_train/num_train, 100 * correct_train/num_train,
      loss_val/num_val, 100 * correct_val/num_val))
  

  # if loss_val < best_loss:
  #   best_loss = loss_val
  #   torch.save(model, os.path.join(base_path, 'audios', "bl-efficient-xl.pth"))
  
  if correct_val > best_acc:
    best_acc = correct_val
    torch.save(model.state_dict(), os.path.join(my_save_path, 
                                              'audios', 
                                              'RGBD',
                                              "mobile-mass{:.2f}.pth".format(100 * correct_val/num_val)))



Using device: cuda
1/100 train loss:0.5667 train acc:37.74% val loss:0.5231 val acc:44.64%
2/100 train loss:0.5083 train acc:43.19% val loss:0.5097 val acc:47.50%
3/100 train loss:0.4908 train acc:46.11% val loss:0.5032 val acc:48.44%
4/100 train loss:0.4784 train acc:47.61% val loss:0.4909 val acc:49.17%
5/100 train loss:0.4683 train acc:49.29% val loss:0.4888 val acc:50.77%
6/100 train loss:0.4605 train acc:51.14% val loss:0.4803 val acc:51.59%
7/100 train loss:0.4529 train acc:52.43% val loss:0.4681 val acc:52.47%
8/100 train loss:0.4413 train acc:53.27% val loss:0.4599 val acc:52.16%
9/100 train loss:0.4282 train acc:54.98% val loss:0.4517 val acc:53.61%
10/100 train loss:0.4172 train acc:55.60% val loss:0.4335 val acc:56.07%
11/100 train loss:0.4110 train acc:56.66% val loss:0.4307 val acc:56.48%
12/100 train loss:0.3991 train acc:58.20% val loss:0.4280 val acc:56.90%
13/100 train loss:0.3940 train acc:59.06% val loss:0.4195 val acc:57.86%
14/100 train loss:0.3837 train acc:60.33%

## height

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

my_save_path = '/content/drive/MyDrive/COSRMAL_CHALLENGE'
bs = 30
train_split = 0.8
lr = 1e-3
epochs = 100
train_set = MiniDataset('/content/train/',
                        '/content/train/labels', 
                        '/content/train/crops_depth',
                        '/content/train/crops_rgb',
                        ['height']
                        )
val_set = MiniDataset('/content/test/',
                      '/content/test/labels', 
                      '/content/test/crops_depth',
                      '/content/test/crops_rgb',
                      ['height']
                       
                      )
model = MobileNetV3_Large(num_classes=1).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr,  weight_decay=1e-5)

best_loss = float('inf')
best_acc = 0

num_train = len(train_set)
num_val = len(val_set)





train_loader   = DataLoader(train_set,
                            batch_size=bs,
                            shuffle=True,
                            num_workers=1,
                            drop_last=True)
val_loader   = DataLoader(val_set,
                          batch_size=bs,
                          shuffle=True,
                          num_workers=1)

for epoch in range(epochs):
  #start_time = time.time()
  loss_train, correct_train = train_image(model, train_loader, optimizer, device, criterion = myLoss)
  loss_val, correct_val = evaluate_image(model, val_loader, device, myLoss)
  #elapsed_time = time.time() - start_time
  print("{}/{} train loss:{:.4f} train acc:{:.2f}% val loss:{:.4f} val acc:{:.2f}%".format(
      epoch+1,epochs, loss_train/num_train, 100 * correct_train/num_train,
      loss_val/num_val, 100 * correct_val/num_val))
  

  # if loss_val < best_loss:
  #   best_loss = loss_val
  #   torch.save(model, os.path.join(base_path, 'audios', "bl-efficient-xl.pth"))
  
  if correct_val > best_acc:
    best_acc = correct_val
    torch.save(model.state_dict(), os.path.join(my_save_path, 
                                              'audios', 
                                              'RGBD',
                                              "mobile-height{:.2f}.pth".format(100 * correct_val/num_val)))



Using device: cuda
1/100 train loss:0.5241 train acc:34.56% val loss:0.6423 val acc:14.70%
2/100 train loss:0.3415 train acc:58.34% val loss:0.7800 val acc:0.76%
3/100 train loss:0.2358 train acc:74.05% val loss:0.2700 val acc:72.25%
4/100 train loss:0.1861 train acc:80.48% val loss:0.1998 val acc:80.02%
5/100 train loss:0.1459 train acc:84.69% val loss:0.2061 val acc:79.39%
6/100 train loss:0.1383 train acc:85.53% val loss:0.1615 val acc:83.85%
7/100 train loss:0.1295 train acc:86.39% val loss:0.1569 val acc:84.31%
8/100 train loss:0.1238 train acc:86.97% val loss:0.2292 val acc:77.08%
9/100 train loss:0.1379 train acc:85.56% val loss:0.1402 val acc:85.98%
10/100 train loss:0.1278 train acc:86.59% val loss:0.1362 val acc:86.38%
11/100 train loss:0.1339 train acc:85.93% val loss:0.2093 val acc:78.79%
12/100 train loss:0.1228 train acc:87.07% val loss:0.1471 val acc:85.29%
13/100 train loss:0.1266 train acc:86.69% val loss:0.1381 val acc:86.19%
14/100 train loss:0.1209 train acc:87.27% 

KeyboardInterrupt: ignored

In [ ]:
!nvidia-smi

## Width top

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

my_save_path = '/content/drive/MyDrive/COSRMAL_CHALLENGE'
bs = 30
train_split = 0.8
lr = 1e-3
epochs = 100
train_set = MiniDataset('/content/train/',
                        '/content/train/labels', 
                        '/content/train/crops_depth',
                        '/content/train/crops_rgb',
                        ['width at the top']
                        )
val_set = MiniDataset('/content/test/',
                      '/content/test/labels', 
                      '/content/test/crops_depth',
                      '/content/test/crops_rgb',
                      ['width at the top']
                       
                      )
model = MobileNetV3_Large(num_classes=1).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr,  weight_decay=1e-5)

best_loss = float('inf')
best_acc = 0

num_train = len(train_set)
num_val = len(val_set)





train_loader   = DataLoader(train_set,
                            batch_size=bs,
                            shuffle=True,
                            num_workers=1,
                            drop_last=True)
val_loader   = DataLoader(val_set,
                          batch_size=bs,
                          shuffle=True,
                          num_workers=1)

for epoch in range(epochs):
  #start_time = time.time()
  loss_train, correct_train = train_image(model, train_loader, optimizer, device, criterion = myLoss)
  loss_val, correct_val = evaluate_image(model, val_loader, device, myLoss)
  #elapsed_time = time.time() - start_time
  print("{}/{} train loss:{:.4f} train acc:{:.2f}% val loss:{:.4f} val acc:{:.2f}%".format(
      epoch+1,epochs, loss_train/num_train, 100 * correct_train/num_train,
      loss_val/num_val, 100 * correct_val/num_val))
  

  # if loss_val < best_loss:
  #   best_loss = loss_val
  #   torch.save(model, os.path.join(base_path, 'audios', "bl-efficient-xl.pth"))
  
  if correct_val > best_acc:
    best_acc = correct_val
    torch.save(model.state_dict(), os.path.join(my_save_path, 
                                              'audios', 
                                              'RGBD',
                                              "mobile-wt{:.2f}.pth".format(100 * correct_val/num_val)))



Using device: cuda
1/100 train loss:0.5314 train acc:39.50% val loss:0.6005 val acc:23.41%
2/100 train loss:0.2665 train acc:69.09% val loss:0.6406 val acc:34.77%
3/100 train loss:0.1906 train acc:79.53% val loss:0.5340 val acc:44.57%
4/100 train loss:0.1843 train acc:80.41% val loss:0.2215 val acc:77.22%
5/100 train loss:0.1499 train acc:84.31% val loss:0.3014 val acc:69.86%
6/100 train loss:0.1405 train acc:85.28% val loss:0.1717 val acc:82.83%
7/100 train loss:0.1491 train acc:84.38% val loss:0.5779 val acc:42.21%
8/100 train loss:0.1401 train acc:85.34% val loss:0.1335 val acc:86.65%
9/100 train loss:0.1587 train acc:83.36% val loss:0.1406 val acc:85.94%
10/100 train loss:0.1332 train acc:86.04% val loss:0.1495 val acc:85.05%
11/100 train loss:0.1254 train acc:86.82% val loss:0.1280 val acc:87.20%
12/100 train loss:0.1159 train acc:87.76% val loss:0.1435 val acc:85.65%
13/100 train loss:0.1275 train acc:86.57% val loss:0.3790 val acc:62.10%
14/100 train loss:0.1245 train acc:86.88%

## Width bottom

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

my_save_path = '/content/drive/MyDrive/COSRMAL_CHALLENGE'
bs = 30
train_split = 0.8
lr = 1e-3
epochs = 100
train_set = MiniDataset('/content/train/',
                        '/content/train/labels', 
                        '/content/train/crops_depth',
                        '/content/train/crops_rgb',
                        ['width at the bottom']
                        )
val_set = MiniDataset('/content/test/',
                      '/content/test/labels', 
                      '/content/test/crops_depth',
                      '/content/test/crops_rgb',
                      ['width at the bottom']
                       
                      )
model = MobileNetV3_Large(num_classes=1).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr,  weight_decay=1e-5)

best_loss = float('inf')
best_acc = 0

num_train = len(train_set)
num_val = len(val_set)





train_loader   = DataLoader(train_set,
                            batch_size=bs,
                            shuffle=True,
                            num_workers=1,
                            drop_last=True)
val_loader   = DataLoader(val_set,
                          batch_size=bs,
                          shuffle=True,
                          num_workers=1)

for epoch in range(epochs):
  #start_time = time.time()
  loss_train, correct_train = train_image(model, train_loader, optimizer, device, criterion = myLoss)
  loss_val, correct_val = evaluate_image(model, val_loader, device, myLoss)
  #elapsed_time = time.time() - start_time
  print("{}/{} train loss:{:.4f} train acc:{:.2f}% val loss:{:.4f} val acc:{:.2f}%".format(
      epoch+1,epochs, loss_train/num_train, 100 * correct_train/num_train,
      loss_val/num_val, 100 * correct_val/num_val))
  

  # if loss_val < best_loss:
  #   best_loss = loss_val
  #   torch.save(model, os.path.join(base_path, 'audios', "bl-efficient-xl.pth"))
  
  if correct_val > best_acc:
    best_acc = correct_val
    torch.save(model.state_dict(), os.path.join(my_save_path, 
                                              'audios', 
                                              'RGBD',
                                              "mobile-wb{:.2f}.pth".format(100 * correct_val/num_val)))



Using device: cuda
1/100 train loss:0.4981 train acc:40.20% val loss:0.3756 val acc:55.33%
2/100 train loss:0.2650 train acc:69.72% val loss:0.2494 val acc:73.34%
3/100 train loss:0.2077 train acc:77.56% val loss:0.8076 val acc:1.79%
4/100 train loss:0.1845 train acc:80.56% val loss:0.6332 val acc:34.38%
5/100 train loss:0.1622 train acc:82.80% val loss:0.1551 val acc:84.49%
6/100 train loss:0.1621 train acc:82.82% val loss:0.1393 val acc:85.95%
7/100 train loss:0.1503 train acc:83.98% val loss:0.3631 val acc:61.43%
8/100 train loss:0.1444 train acc:84.65% val loss:0.1260 val acc:87.15%
9/100 train loss:0.1362 train acc:85.55% val loss:0.1376 val acc:86.24%
10/100 train loss:0.1363 train acc:85.52% val loss:0.1353 val acc:86.47%
11/100 train loss:0.1459 train acc:84.48% val loss:0.1963 val acc:80.37%
12/100 train loss:0.1396 train acc:85.22% val loss:0.1243 val acc:87.30%
13/100 train loss:0.1306 train acc:86.06% val loss:0.1328 val acc:86.59%
14/100 train loss:0.1272 train acc:86.48% 

## Transfer

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

my_save_path = '/content/drive/MyDrive/COSRMAL_CHALLENGE'
bs = 30
train_split = 0.8
lr = 1e-3
epochs = 100
train_set = MiniDataset('/content/train/',
                        '/content/train/labels', 
                        '/content/train/crops_depth',
                        '/content/train/crops_rgb')
val_set = MiniDataset('/content/test/',
                        '/content/test/labels', 
                        '/content/test/crops_depth',
                        '/content/test/crops_rgb')
model = MobileNetV3_Large(num_classes=1)
pretrain = '/content/drive/MyDrive/COSRMAL_CHALLENGE/audios/RGBD/mobile-wt88.78.pth'
model.load_state_dict(torch.load(pretrain))
model.to(device)
optimizer = optim.Adam(model.parameters(), lr=lr,  weight_decay=1e-5)

best_loss = float('inf')
best_acc = 0

num_train = len(train_set)
num_val = len(val_set)





train_loader   = DataLoader(train_set,
                            batch_size=bs,
                            shuffle=True,
                            num_workers=1,
                            drop_last=True)
val_loader   = DataLoader(val_set,
                          batch_size=bs,
                          shuffle=True,
                          num_workers=1)

for epoch in range(epochs):
  #start_time = time.time()
  loss_train, correct_train = train_image(model, train_loader, optimizer, device, criterion = myLoss)
  loss_val, correct_val = evaluate_image(model, val_loader, device, myLoss)
  #elapsed_time = time.time() - start_time
  print("{}/{} train loss:{:.4f} train acc:{:.2f}% val loss:{:.4f} val acc:{:.2f}%".format(
      epoch+1,epochs, loss_train/num_train, 100 * correct_train/num_train,
      loss_val/num_val, 100 * correct_val/num_val))
  

  # if loss_val < best_loss:
  #   best_loss = loss_val
  #   torch.save(model, os.path.join(base_path, 'audios', "bl-efficient-xl.pth"))
  
  if correct_val > best_acc:
    best_acc = correct_val
    torch.save(model.state_dict(), os.path.join(my_save_path, 
                                              'audios', 
                                              'RGBD',
                                              "mobile{:.2f}.pth".format(100 * correct_val/num_val)))



Using device: cuda
1/100 train loss:0.3746 train acc:57.25% val loss:0.3028 val acc:68.45%
2/100 train loss:0.2891 train acc:67.07% val loss:0.2930 val acc:70.70%
3/100 train loss:0.2518 train acc:72.43% val loss:0.5375 val acc:41.42%
4/100 train loss:0.2243 train acc:75.84% val loss:0.4622 val acc:53.36%
5/100 train loss:0.2000 train acc:79.29% val loss:0.2390 val acc:76.10%
6/100 train loss:0.1855 train acc:80.82% val loss:0.3805 val acc:61.11%
7/100 train loss:0.1857 train acc:80.79% val loss:0.2686 val acc:73.14%
8/100 train loss:0.1731 train acc:82.03% val loss:0.2466 val acc:75.34%
9/100 train loss:0.1703 train acc:82.31% val loss:0.2486 val acc:75.14%
10/100 train loss:0.1826 train acc:81.02% val loss:0.3201 val acc:67.99%
11/100 train loss:0.1669 train acc:82.62% val loss:0.2178 val acc:78.22%
12/100 train loss:0.1493 train acc:84.40% val loss:0.2141 val acc:78.34%
13/100 train loss:0.1418 train acc:85.17% val loss:0.2774 val acc:72.26%
14/100 train loss:0.1373 train acc:85.62%

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

my_save_path = '/content/drive/MyDrive/COSRMAL_CHALLENGE'
bs = 30
train_split = 0.8
lr = 1e-4
epochs = 200
train_set = MiniDataset('/content/train/',
                        '/content/train/labels', 
                        '/content/train/crops_depth',
                        '/content/train/crops_rgb',
                        ['container mass']
                        )
val_set = MiniDataset('/content/test/',
                      '/content/test/labels', 
                      '/content/test/crops_depth',
                      '/content/test/crops_rgb',
                      ['container mass']          
                      )
model = MobileNetV3_Large(num_classes=1)
pretrain = '/content/drive/MyDrive/COSRMAL_CHALLENGE/audios/RGBD/mobile-wt88.78.pth'
model.load_state_dict(torch.load(pretrain))
model.to(device)
optimizer = optim.Adam(model.parameters(), lr=lr,  weight_decay=1e-5)

best_loss = float('inf')
best_acc = 0

num_train = len(train_set)
num_val = len(val_set)





train_loader   = DataLoader(train_set,
                            batch_size=bs,
                            shuffle=True,
                            num_workers=1,
                            drop_last=True)
val_loader   = DataLoader(val_set,
                          batch_size=bs,
                          shuffle=True,
                          num_workers=1)

for epoch in range(epochs):
  #start_time = time.time()
  loss_train, correct_train = train_image(model, train_loader, optimizer, device, criterion = myLoss)
  loss_val, correct_val = evaluate_image(model, val_loader, device, myLoss)
  #elapsed_time = time.time() - start_time
  print("{}/{} train loss:{:.4f} train acc:{:.2f}% val loss:{:.4f} val acc:{:.2f}%".format(
      epoch+1,epochs, loss_train/num_train, 100 * correct_train/num_train,
      loss_val/num_val, 100 * correct_val/num_val))
  

  # if loss_val < best_loss:
  #   best_loss = loss_val
  #   torch.save(model, os.path.join(base_path, 'audios', "bl-efficient-xl.pth"))
  
  if correct_val > best_acc:
    best_acc = correct_val
    torch.save(model.state_dict(), os.path.join(my_save_path, 
                                              'audios', 
                                              'RGBD',
                                              "mobile-mass{:.2f}.pth".format(100 * correct_val/num_val)))



Using device: cuda
1/200 train loss:0.7068 train acc:28.62% val loss:0.6739 val acc:32.33%
2/200 train loss:0.6874 train acc:30.53% val loss:0.6544 val acc:34.56%
3/200 train loss:0.6820 train acc:31.10% val loss:0.6364 val acc:36.36%
4/200 train loss:0.6744 train acc:31.85% val loss:0.6362 val acc:36.38%
5/200 train loss:0.6648 train acc:32.74% val loss:0.6391 val acc:35.65%
6/200 train loss:0.6571 train acc:33.54% val loss:0.6412 val acc:35.59%
7/200 train loss:0.6421 train acc:35.06% val loss:0.6135 val acc:38.65%
8/200 train loss:0.6359 train acc:35.60% val loss:0.6261 val acc:37.39%
9/200 train loss:0.6373 train acc:35.43% val loss:0.5957 val acc:40.43%
10/200 train loss:0.6311 train acc:36.15% val loss:0.6008 val acc:39.92%
11/200 train loss:0.6232 train acc:36.84% val loss:0.5813 val acc:41.61%
12/200 train loss:0.6133 train acc:37.92% val loss:0.5554 val acc:44.22%
13/200 train loss:0.6044 train acc:38.81% val loss:0.5662 val acc:43.38%
14/200 train loss:0.5993 train acc:39.37%

## Augmentation

In [ ]:
def randomlyAug(crop, depth, label, max_val=640):
  h, w, c = crop.shape

  if h >= w:
    max_dim = h
  else:
    max_dim = w
  max_rand = max_val / max_dim

  rand_num = np.random.uniform(0.5,max_rand,1).item()

  width = int(w * rand_num)
  height = int(h * rand_num)
  dim = (width, height)
    
  # resize image
  crop = cv2.resize(crop, dim, interpolation = cv2.INTER_AREA)
  depth = cv2.resize(depth, dim, interpolation = cv2.INTER_NEAREST)[:, :, np.newaxis]

  label *= (height / h)

  
  
  return crop, depth, label
 



def get_annotation(id,input,anno_path='/content/labels'):
    anno = np.load(os.path.join(anno_path,'{:06d}.npy'.format(id)),allow_pickle=True).item()
    return anno.get(input)
    

class MiniDataset(Dataset):
    def __init__(self, base_p, label_f, depth, crops_rgb_f, aug=False, label_name=['container capacity']):
      self.label_f = label_f #_f = folder
      self.depth = depth
      self.base = base_p
      self.label_name = label_name
      self.crops_rgb_f = crops_rgb_f
      self.samples = os.listdir(crops_rgb_f)
      self.ids = [ int(x.split('.')[0]) for x in self.samples]
      self.transform = transforms.Compose([
                                             transforms.Resize((320, 320)),
                                             transforms.ToTensor(),
                                             transforms.ConvertImageDtype(torch.float),
                                             ])
      self.aug = aug
    def __len__(self):
      return len(self.ids)

    def __getitem__(self, idx):
      id_ = self.ids[idx]
        
      # depth
      depth = np.asarray(Image.open(os.path.join(self.depth,'{:06d}.png'.format(id_))))[:,:,np.newaxis]
      
      # rgb_cropped
      crop = np.asarray(Image.open(os.path.join(self.crops_rgb_f,'{:06d}.png'.format(id_))))
      # label
      label = np.array([get_annotation(id_,name,os.path.join(self.base, 'labels')) for name in self.label_name])

      if self.aug:
          crop, depth, label = randomlyAug(crop, depth, label, max_val=640)

      h, w, c = crop.shape

      resX = 640 - h
      resY = 640 - w

      up = resX // 2
      down = up
      if resX % 2 != 0:
        down +=1

      left = resY // 2
      right = left

      if resY % 2 != 0:
        left += 1

      padding = transforms.Pad((left, up, right, down))

    
      image = Image.fromarray(np.concatenate((crop, depth), axis=2))
      image = padding(image)
      image = self.transform(image)
      

      return image, label

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

my_save_path = '/content/drive/MyDrive/COSRMAL_CHALLENGE'
bs = 30
train_split = 0.8
lr = 1e-3
epochs = 100
train_set = MiniDataset('/content/train/',
                        '/content/train/labels', 
                        '/content/train/crops_depth',
                        '/content/train/crops_rgb',
                        aug=True)
val_set = MiniDataset('/content/test/',
                        '/content/test/labels', 
                        '/content/test/crops_depth',
                        '/content/test/crops_rgb')
model = MobileNetV3_Large(num_classes=1).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr,  weight_decay=1e-5)

best_loss = float('inf')
best_acc = 0

num_train = len(train_set)
num_val = len(val_set)





train_loader   = DataLoader(train_set,
                            batch_size=bs,
                            shuffle=True,
                            num_workers=1,
                            drop_last=True)
val_loader   = DataLoader(val_set,
                          batch_size=bs,
                          shuffle=True,
                          num_workers=1)

for epoch in range(epochs):
  #start_time = time.time()
  loss_train, correct_train = train_image(model, train_loader, optimizer, device, criterion = myLoss)
  loss_val, correct_val = evaluate_image(model, val_loader, device, myLoss)
  #elapsed_time = time.time() - start_time
  print("{}/{} train loss:{:.4f} train acc:{:.2f}% val loss:{:.4f} val acc:{:.2f}%".format(
      epoch+1,epochs, loss_train/num_train, 100 * correct_train/num_train,
      loss_val/num_val, 100 * correct_val/num_val))
  

  # if loss_val < best_loss:
  #   best_loss = loss_val
  #   torch.save(model, os.path.join(base_path, 'audios', "bl-efficient-xl.pth"))
  
  if correct_val > best_acc:
    best_acc = correct_val
    torch.save(model.state_dict(), os.path.join(my_save_path, 
                                              'audios', 
                                              'RGBD',
                                              "mobile-aug{:.2f}.pth".format(100 * correct_val/num_val)))



Using device: cuda
1/100 train loss:0.6046 train acc:21.23% val loss:0.5439 val acc:37.81%
2/100 train loss:0.5463 train acc:29.05% val loss:0.3676 val acc:62.11%
3/100 train loss:0.5021 train acc:38.07% val loss:0.3545 val acc:63.48%
4/100 train loss:0.4593 train acc:44.55% val loss:0.3810 val acc:58.59%
5/100 train loss:0.4084 train acc:52.29% val loss:0.3735 val acc:62.35%
6/100 train loss:0.3723 train acc:58.19% val loss:0.3151 val acc:68.20%
7/100 train loss:0.3370 train acc:62.11% val loss:0.2992 val acc:70.08%
8/100 train loss:0.2984 train acc:67.44% val loss:0.7842 val acc:21.58%
9/100 train loss:0.2751 train acc:71.59% val loss:0.2216 val acc:77.84%
10/100 train loss:0.2663 train acc:72.74% val loss:0.4636 val acc:53.64%
11/100 train loss:0.2620 train acc:73.17% val loss:0.2832 val acc:71.68%
12/100 train loss:0.2506 train acc:74.30% val loss:0.2553 val acc:74.47%
13/100 train loss:0.2387 train acc:75.49% val loss:0.2207 val acc:77.93%
14/100 train loss:0.2356 train acc:75.80%

## MobileCA

In [ ]:
from models import mbv2_ca

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

my_save_path = '/content/drive/MyDrive/COSRMAL_CHALLENGE'
bs = 30
train_split = 0.8
lr = 1e-3
epochs = 100
train_set = MiniDataset('/content/train/',
                        '/content/train/labels', 
                        '/content/train/crops_depth',
                        '/content/train/crops_rgb',
                        aug=True
                        )
val_set = MiniDataset('/content/test/',
                      '/content/test/labels', 
                      '/content/test/crops_depth',
                      '/content/test/crops_rgb',
                       
                      )
model = mbv2_ca(num_classes=1).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr,  weight_decay=1e-5)

best_loss = float('inf')
best_acc = 0

num_train = len(train_set)
num_val = len(val_set)





train_loader   = DataLoader(train_set,
                            batch_size=bs,
                            shuffle=True,
                            num_workers=1,
                            drop_last=True)
val_loader   = DataLoader(val_set,
                          batch_size=bs,
                          shuffle=True,
                          num_workers=1)

for epoch in range(epochs):
  #start_time = time.time()
  loss_train, correct_train = train_image(model, train_loader, optimizer, device, criterion = myLoss)
  loss_val, correct_val = evaluate_image(model, val_loader, device, myLoss)
  #elapsed_time = time.time() - start_time
  print("{}/{} train loss:{:.4f} train acc:{:.2f}% val loss:{:.4f} val acc:{:.2f}%".format(
      epoch+1,epochs, loss_train/num_train, 100 * correct_train/num_train,
      loss_val/num_val, 100 * correct_val/num_val))
  

  # if loss_val < best_loss:
  #   best_loss = loss_val
  #   torch.save(model, os.path.join(base_path, 'audios', "bl-efficient-xl.pth"))
  
  if correct_val > best_acc:
    best_acc = correct_val
    torch.save(model.state_dict(), os.path.join(my_save_path, 
                                              'audios', 
                                              'RGBD',
                                              "mobile-ca{:.2f}.pth".format(100 * correct_val/num_val)))



Using device: cuda
1/100 train loss:0.5660 train acc:42.75% val loss:0.6299 val acc:37.01%
2/100 train loss:0.4838 train acc:50.99% val loss:0.5819 val acc:41.81%
3/100 train loss:0.4136 train acc:58.01% val loss:0.3188 val acc:68.12%
4/100 train loss:0.3301 train acc:66.36% val loss:0.4220 val acc:57.80%
5/100 train loss:0.2672 train acc:72.65% val loss:0.7110 val acc:28.90%
6/100 train loss:0.2441 train acc:74.96% val loss:0.2595 val acc:74.05%
7/100 train loss:0.2269 train acc:76.67% val loss:0.3324 val acc:66.76%
8/100 train loss:0.2166 train acc:77.71% val loss:0.2413 val acc:75.87%
9/100 train loss:0.2116 train acc:78.21% val loss:0.2650 val acc:73.50%
10/100 train loss:0.2103 train acc:78.33% val loss:0.2533 val acc:74.67%
11/100 train loss:0.2008 train acc:79.29% val loss:0.2437 val acc:75.63%
12/100 train loss:0.1962 train acc:79.75% val loss:0.3192 val acc:68.08%
13/100 train loss:0.1909 train acc:80.27% val loss:0.2448 val acc:75.52%
14/100 train loss:0.1920 train acc:80.16%

## Chaoran dataset

In [ ]:
def randomlyAug(crop, depth, label, max_val=640):
  h, w, c = crop.shape

  if h >= w:
    max_dim = h
  else:
    max_dim = w
  max_rand = max_val / max_dim

  rand_num = np.random.uniform(0.5,max_rand,1).item()

  width = int(w * rand_num)
  height = int(h * rand_num)
  dim = (width, height)
    
  # resize image
  crop = cv2.resize(crop, dim, interpolation = cv2.INTER_AREA)
  depth = cv2.resize(depth, dim, interpolation = cv2.INTER_NEAREST)[:, :, np.newaxis]

  label *= (height / h)

  
  
  return crop, depth, label
 



def get_annotation(id,input,anno_path='/content/labels'):
    anno = np.load(os.path.join(anno_path,'{:06d}.npy'.format(id)),allow_pickle=True).item()
    return anno.get(input)
    

class MiniDataset(Dataset):
    def __init__(self, base_p, label_f, depth, crops_rgb_f, aug=False, label_name=['container capacity']):
      self.label_f = label_f #_f = folder
      self.depth = depth
      self.base = base_p
      self.label_name = label_name
      self.crops_rgb_f = crops_rgb_f
      self.samples = os.listdir(crops_rgb_f)
      self.ids = [ int(x.split('.')[0]) for x in self.samples]
      self.transform = transforms.Compose([
                                             transforms.Resize((320, 320)),
                                             transforms.ToTensor(),
                                             transforms.ConvertImageDtype(torch.float),
                                             ])
      self.aug = aug
    def __len__(self):
      return len(self.ids)

    def __getitem__(self, idx):
      id_ = self.ids[idx]
        
      # depth
      depth = np.asarray(Image.open(os.path.join(self.depth,'{:06d}.png'.format(id_))))[:,:,np.newaxis]
      
      # rgb_cropped
      crop = np.asarray(Image.open(os.path.join(self.crops_rgb_f,'{:06d}.png'.format(id_))))
      # label
      label = np.array([get_annotation(id_,name,os.path.join(self.base, 'labels')) for name in self.label_name])

      if self.aug:
          crop, depth, label = randomlyAug(crop, depth, label, max_val=640)

      h, w, c = crop.shape

      resX = 640 - h
      resY = 640 - w

      up = resX // 2
      down = up
      if resX % 2 != 0:
        down +=1

      left = resY // 2
      right = left

      if resY % 2 != 0:
        left += 1

      padding = transforms.Pad((left, up, right, down))

    
      image = Image.fromarray(np.concatenate((crop, depth), axis=2))
      image = padding(image)
      image = self.transform(image)
      

      return image, label

In [ ]:
from models import mbv2_ca

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

my_save_path = '/content/drive/MyDrive/COSRMAL_CHALLENGE/task3'
bs = 30
train_split = 0.8
lr = 1e-3
epochs = 100
train_set = MiniDataset('/content/train/',
                        '/content/train/labels', 
                        '/content/train/crops_depth',
                        '/content/train/crops_rgb',
                        aug=True
                        )
val_set = MiniDataset('/content/test/',
                      '/content/test/labels', 
                      '/content/test/crops_depth',
                      '/content/test/crops_rgb',
                       
                      )
model = mbv2_ca(num_classes=1).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr,  weight_decay=1e-5)

best_loss = float('inf')
best_acc = 0

num_train = len(train_set)
num_val = len(val_set)





train_loader   = DataLoader(train_set,
                            batch_size=bs,
                            shuffle=True,
                            num_workers=1,
                            drop_last=True)
val_loader   = DataLoader(val_set,
                          batch_size=bs,
                          shuffle=True,
                          num_workers=1)

for epoch in range(epochs):
  #start_time = time.time()
  loss_train, correct_train = train_image(model, train_loader, optimizer, device, criterion = myLoss)
  loss_val, correct_val = evaluate_image(model, val_loader, device, myLoss)
  #elapsed_time = time.time() - start_time
  print("{}/{} train loss:{:.4f} train acc:{:.2f}% val loss:{:.4f} val acc:{:.2f}%".format(
      epoch+1,epochs, loss_train/num_train, 100 * correct_train/num_train,
      loss_val/num_val, 100 * correct_val/num_val))
  

  # if loss_val < best_loss:
  #   best_loss = loss_val
  #   torch.save(model, os.path.join(base_path, 'audios', "bl-efficient-xl.pth"))
  
  if correct_val > best_acc:
    best_acc = correct_val
    torch.save(model.state_dict(), os.path.join(my_save_path, 
                                              "mobile-ca{:.2f}.pth".format(100 * correct_val/num_val)))



Using device: cuda
1/100 train loss:0.5757 train acc:42.17% val loss:0.6286 val acc:37.14%
2/100 train loss:0.4836 train acc:51.40% val loss:0.4326 val acc:56.74%
3/100 train loss:0.4223 train acc:57.54% val loss:0.5826 val acc:41.74%
4/100 train loss:0.3646 train acc:63.30% val loss:0.4875 val acc:51.25%
5/100 train loss:0.3035 train acc:69.42% val loss:0.1998 val acc:80.02%
6/100 train loss:0.2727 train acc:72.49% val loss:0.3511 val acc:64.89%
7/100 train loss:0.2281 train acc:76.96% val loss:0.4130 val acc:58.70%
8/100 train loss:0.2076 train acc:79.00% val loss:0.1869 val acc:81.31%
9/100 train loss:0.1834 train acc:81.43% val loss:0.2043 val acc:79.57%
10/100 train loss:0.1797 train acc:81.79% val loss:0.3986 val acc:60.14%
11/100 train loss:0.1731 train acc:82.45% val loss:0.3703 val acc:62.97%
12/100 train loss:0.1718 train acc:82.58% val loss:0.3273 val acc:67.27%
13/100 train loss:0.1646 train acc:83.30% val loss:0.4105 val acc:58.95%
14/100 train loss:0.1577 train acc:83.99%

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

my_save_path = '/content/drive/MyDrive/COSRMAL_CHALLENGE/task3/mobileV3'
bs = 30
train_split = 0.8
lr = 1e-3
epochs = 100
train_set = MiniDataset('/content/train/',
                        '/content/train/labels', 
                        '/content/train/crops_depth',
                        '/content/train/crops_rgb',
                        aug=True)
val_set = MiniDataset('/content/test/',
                        '/content/test/labels', 
                        '/content/test/crops_depth',
                        '/content/test/crops_rgb')
model = MobileNetV3_Large(num_classes=1).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr,  weight_decay=1e-5)

best_loss = float('inf')
best_acc = 0

num_train = len(train_set)
num_val = len(val_set)





train_loader   = DataLoader(train_set,
                            batch_size=bs,
                            shuffle=True,
                            num_workers=1,
                            drop_last=True)
val_loader   = DataLoader(val_set,
                          batch_size=bs,
                          shuffle=True,
                          num_workers=1)

for epoch in range(epochs):
  #start_time = time.time()
  loss_train, correct_train = train_image(model, train_loader, optimizer, device, criterion = myLoss)
  loss_val, correct_val = evaluate_image(model, val_loader, device, myLoss)
  #elapsed_time = time.time() - start_time
  print("{}/{} train loss:{:.4f} train acc:{:.2f}% val loss:{:.4f} val acc:{:.2f}%".format(
      epoch+1,epochs, loss_train/num_train, 100 * correct_train/num_train,
      loss_val/num_val, 100 * correct_val/num_val))
  

  # if loss_val < best_loss:
  #   best_loss = loss_val
  #   torch.save(model, os.path.join(base_path, 'audios', "bl-efficient-xl.pth"))
  
  if correct_val > best_acc:
    best_acc = correct_val
    torch.save(model.state_dict(), os.path.join(my_save_path,
                                              "mobile-aug{:.2f}.pth".format(100 * correct_val/num_val)))



Using device: cuda
1/100 train loss:0.6007 train acc:18.98% val loss:0.5936 val acc:37.28%
2/100 train loss:0.5425 train acc:26.85% val loss:0.5103 val acc:48.29%
3/100 train loss:0.4951 train acc:35.74% val loss:0.4270 val acc:57.03%
4/100 train loss:0.4432 train acc:44.76% val loss:0.5623 val acc:29.87%
5/100 train loss:0.4003 train acc:51.15% val loss:0.6894 val acc:0.30%
6/100 train loss:0.3715 train acc:55.96% val loss:0.2282 val acc:77.18%
7/100 train loss:0.3684 train acc:60.66% val loss:0.2483 val acc:75.17%
8/100 train loss:0.3058 train acc:67.16% val loss:0.4231 val acc:57.69%
9/100 train loss:0.2833 train acc:70.18% val loss:0.3984 val acc:60.02%
10/100 train loss:0.2671 train acc:72.29% val loss:0.3780 val acc:62.20%
11/100 train loss:0.2417 train acc:75.38% val loss:0.3536 val acc:64.64%
12/100 train loss:0.2277 train acc:76.89% val loss:0.3453 val acc:65.47%
13/100 train loss:0.2184 train acc:77.88% val loss:0.3924 val acc:60.76%
14/100 train loss:0.2277 train acc:76.86% 

In [ ]:
weights = torch.load('mbv2_ca.pth')

In [ ]:
model.features[0][0]

Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)

## MobileV2 pretrained weights

## Capacity

In [ ]:
from models import mbv2_ca

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

my_save_path = '/content/drive/MyDrive/COSRMAL_CHALLENGE/task3/mobile_ca_pretrain'
os.makedirs(my_save_path,exist_ok=True)

bs = 30
train_split = 0.8
lr = 1e-3
epochs = 100
train_set = MiniDataset('/content/train/',
                        '/content/train/labels', 
                        '/content/train/crops_depth',
                        '/content/train/crops_rgb',
                        aug=True
                        )
val_set = MiniDataset('/content/test/',
                      '/content/test/labels', 
                      '/content/test/crops_depth',
                      '/content/test/crops_rgb',
                       
                      )
model = mbv2_ca(in_c=3, num_classes=1).to(device)

weights = torch.load('mbv2_ca.pth')
model.load_state_dict(weights, strict=False)
pretrained_weights = model.features[0][0].weight
model.features[0][0] = nn.Conv2d(4, 32, kernel_size=3, stride=2, padding=1, bias=False)
model.features[0][0].weight.data.normal_(0, 0.001)
model.features[0][0].weight.data[:, :3, :, :] = pretrained_weights

model.to(device)
optimizer = optim.Adam(model.parameters(), lr=lr,  weight_decay=1e-5)

best_loss = float('inf')
best_acc = 0

num_train = len(train_set)
num_val = len(val_set)





train_loader   = DataLoader(train_set,
                            batch_size=bs,
                            shuffle=True,
                            num_workers=1,
                            drop_last=True)
val_loader   = DataLoader(val_set,
                          batch_size=bs,
                          shuffle=True,
                          num_workers=1)

for epoch in range(epochs):
  #start_time = time.time()
  loss_train, correct_train = train_image(model, train_loader, optimizer, device, criterion = myLoss)
  loss_val, correct_val = evaluate_image(model, val_loader, device, myLoss)
  #elapsed_time = time.time() - start_time
  print("{}/{} train loss:{:.4f} train acc:{:.2f}% val loss:{:.4f} val acc:{:.2f}%".format(
      epoch+1,epochs, loss_train/num_train, 100 * correct_train/num_train,
      loss_val/num_val, 100 * correct_val/num_val))
  

  # if loss_val < best_loss:
  #   best_loss = loss_val
  #   torch.save(model, os.path.join(base_path, 'audios', "bl-efficient-xl.pth"))
  
  if correct_val > best_acc:
    best_acc = correct_val
    torch.save(model.state_dict(), os.path.join(my_save_path, 
                                              "mobile-ca{:.2f}.pth".format(100 * correct_val/num_val)))



Using device: cuda
1/100 train loss:0.5549 train acc:43.06% val loss:0.4770 val acc:52.30%
2/100 train loss:0.4588 train acc:53.89% val loss:0.3632 val acc:63.68%
3/100 train loss:0.3950 train acc:60.26% val loss:0.3361 val acc:66.39%
4/100 train loss:0.3370 train acc:66.06% val loss:0.5445 val acc:45.55%
5/100 train loss:0.2889 train acc:70.88% val loss:0.3780 val acc:62.20%
6/100 train loss:0.2498 train acc:74.78% val loss:0.3302 val acc:66.98%
7/100 train loss:0.2193 train acc:77.83% val loss:0.3735 val acc:62.65%
8/100 train loss:0.1942 train acc:80.35% val loss:0.4059 val acc:59.41%
9/100 train loss:0.1778 train acc:81.98% val loss:0.4124 val acc:58.76%
10/100 train loss:0.1738 train acc:82.38% val loss:0.4171 val acc:58.29%
11/100 train loss:0.1669 train acc:83.08% val loss:0.4141 val acc:58.59%
12/100 train loss:0.1601 train acc:83.76% val loss:0.3806 val acc:61.94%
13/100 train loss:0.1558 train acc:84.18% val loss:0.4167 val acc:58.33%
14/100 train loss:0.1610 train acc:83.66%

In [ ]:
from models import mbv2_ca

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

my_save_path = '/content/drive/MyDrive/COSRMAL_CHALLENGE/task3/mobile_ca_pretrain'
os.makedirs(my_save_path,exist_ok=True)

bs = 30
train_split = 0.8
lr = 1e-3
epochs = 100
train_set = MiniDataset('/content/train/',
                        '/content/train/labels', 
                        '/content/train/crops_depth',
                        '/content/train/crops_rgb',
                        aug=False
                        )
val_set = MiniDataset('/content/test/',
                      '/content/test/labels', 
                      '/content/test/crops_depth',
                      '/content/test/crops_rgb',
                       
                      )
model = mbv2_ca(in_c=3, num_classes=1).to(device)

weights = torch.load('mbv2_ca.pth')
model.load_state_dict(weights, strict=False)
pretrained_weights = model.features[0][0].weight
model.features[0][0] = nn.Conv2d(4, 32, kernel_size=3, stride=2, padding=1, bias=False)
model.features[0][0].weight.data.normal_(0, 0.001)
model.features[0][0].weight.data[:, :3, :, :] = pretrained_weights

model.to(device)
optimizer = optim.Adam(model.parameters(), lr=lr,  weight_decay=1e-5)

best_loss = float('inf')
best_acc = 0

num_train = len(train_set)
num_val = len(val_set)





train_loader   = DataLoader(train_set,
                            batch_size=bs,
                            shuffle=True,
                            num_workers=1,
                            drop_last=True)
val_loader   = DataLoader(val_set,
                          batch_size=bs,
                          shuffle=True,
                          num_workers=1)

for epoch in range(epochs):
  #start_time = time.time()
  loss_train, correct_train = train_image(model, train_loader, optimizer, device, criterion = myLoss)
  loss_val, correct_val = evaluate_image(model, val_loader, device, myLoss)
  #elapsed_time = time.time() - start_time
  print("{}/{} train loss:{:.4f} train acc:{:.2f}% val loss:{:.4f} val acc:{:.2f}%".format(
      epoch+1,epochs, loss_train/num_train, 100 * correct_train/num_train,
      loss_val/num_val, 100 * correct_val/num_val))
  

  # if loss_val < best_loss:
  #   best_loss = loss_val
  #   torch.save(model, os.path.join(base_path, 'audios', "bl-efficient-xl.pth"))
  
  # if correct_val > best_acc:
  #   best_acc = correct_val
  #   torch.save(model.state_dict(), os.path.join(my_save_path, 
  #                                             "mobile-ca{:.2f}.pth".format(100 * correct_val/num_val)))



Using device: cuda
1/100 train loss:0.5191 train acc:47.29% val loss:0.4214 val acc:57.86%
2/100 train loss:0.3619 train acc:63.57% val loss:0.4953 val acc:50.47%
3/100 train loss:0.2869 train acc:71.07% val loss:0.4049 val acc:59.51%
4/100 train loss:0.2364 train acc:76.13% val loss:0.4278 val acc:57.22%
5/100 train loss:0.1912 train acc:80.65% val loss:0.4136 val acc:58.64%
6/100 train loss:0.1400 train acc:85.76% val loss:0.3892 val acc:61.08%
7/100 train loss:0.1176 train acc:88.01% val loss:0.4462 val acc:55.38%
8/100 train loss:0.1159 train acc:88.18% val loss:0.4038 val acc:59.62%
9/100 train loss:0.1003 train acc:89.73% val loss:0.4520 val acc:54.80%
10/100 train loss:0.1032 train acc:89.44% val loss:0.4278 val acc:57.22%
11/100 train loss:0.0978 train acc:89.98% val loss:0.4825 val acc:51.75%
12/100 train loss:0.0941 train acc:90.35% val loss:0.4588 val acc:54.12%
13/100 train loss:0.0936 train acc:90.40% val loss:0.4088 val acc:59.12%
14/100 train loss:0.0966 train acc:90.10%

In [ ]:
import copy
import torch
import numpy as np

def computeScoreType1(gt, _est):
  gt = gt.cpu().data.numpy()
  _est = _est.cpu().data.numpy()
  est = copy.deepcopy(_est)
  assert (len(gt) == len(est))
  # if all(x == -1 for x in est):
  #   return 0
  indicator_f = est > -1
  ec = np.exp(-(np.abs(gt - est) / gt)) * indicator_f
  score = np.sum(ec) / len(gt)
  return score

def myLoss(est, gt):
  ec = 1 - torch.exp(-(torch.abs(gt - est) / gt))
  score = torch.sum(ec) / len(gt)

  return score

def train_image(model, train_loader, optimizer, device, criterion = nn.L1Loss()):
  model.train()
  loss_train = 0.0
  correct_train = 0.0
  num_train = len(train_loader)
  for batch_idx, (audio, target) in enumerate(train_loader):
    audio = audio.to(device)
    target = target.to(device)

    optimizer.zero_grad()
    outputs = model.forward(audio)
    loss = criterion(outputs, target)
    loss.backward()
    #nn.utils.clip_grad_norm_(model.parameters(), 0.5)
    optimizer.step()

    loss_train += loss.item() * audio.shape[0]
    correct_train += computeScoreType1(target, outputs) * audio.shape[0]

    
    
  
  return loss_train, correct_train


def evaluate_image(model, testloader, device, criterion = nn.L1Loss()):
  model.eval()
  loss_test = 0
  correct_test=0
  num_val = len(testloader)
  with torch.no_grad():
    for batch_idx, (audio, target) in enumerate(testloader):
      audio = audio.to(device)
      target = target.to(device)
      outputs = model.forward(audio)
      loss = criterion(outputs, target)
      loss_test += loss.item() * audio.shape[0]
      correct_test += computeScoreType1(target, outputs) * audio.shape[0]
  return loss_test, correct_test



In [ ]:
from models import mbv2_ca

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

my_save_path = '/content/drive/MyDrive/COSRMAL_CHALLENGE/task5/mobile_ca_pretrain'
os.makedirs(my_save_path,exist_ok=True)

bs = 30
train_split = 0.8
lr = 5e-4
epochs = 100
train_set = MiniDataset('/content/train/',
                        '/content/train/labels', 
                        '/content/train/crops_depth',
                        '/content/train/crops_rgb',
                        aug=True,
                        label_name=['width at the top', 'width at the bottom', 'height']
                        )
val_set = MiniDataset('/content/test/',
                      '/content/test/labels', 
                      '/content/test/crops_depth',
                      '/content/test/crops_rgb',
                      label_name=['width at the top', 'width at the bottom', 'height']
                      )
model = mbv2_ca(in_c=3, num_classes=3).to(device)

weights = torch.load('mbv2_ca.pth')
model.load_state_dict(weights, strict=False)
pretrained_weights = model.features[0][0].weight
model.features[0][0] = nn.Conv2d(4, 32, kernel_size=3, stride=2, padding=1, bias=False)
model.features[0][0].weight.data.normal_(0, 0.001)
model.features[0][0].weight.data[:, :3, :, :] = pretrained_weights

model.to(device)
optimizer = optim.Adam(model.parameters(), lr=lr,  weight_decay=1e-5)

best_loss = float('inf')
best_acc = 0

num_train = len(train_set)
num_val = len(val_set)





train_loader   = DataLoader(train_set,
                            batch_size=bs,
                            shuffle=True,
                            num_workers=1,
                            drop_last=True)
val_loader   = DataLoader(val_set,
                          batch_size=bs,
                          shuffle=True,
                          num_workers=1)

for epoch in range(epochs):
  #start_time = time.time()
  loss_train, correct_train = train_image(model, train_loader, optimizer, device, criterion = myLoss)
  loss_val, correct_val = evaluate_image(model, val_loader, device, myLoss)
  #elapsed_time = time.time() - start_time
  print("{}/{} train loss:{:.4f} train acc:{:.2f}% val loss:{:.4f} val acc:{:.2f}%".format(
      epoch+1,epochs, 1/3 * loss_train/num_train, 1/3 * 100 * correct_train/num_train,
      1/3 * loss_val/num_val, 1/3 * 100 * correct_val/num_val))
  

  # if loss_val < best_loss:
  #   best_loss = loss_val
  #   torch.save(model, os.path.join(base_path, 'audios', "bl-efficient-xl.pth"))
  
  if correct_val > best_acc:
    best_acc = correct_val
    torch.save(model.state_dict(), os.path.join(my_save_path, 
                                              "mobile-ca{:.2f}.pth".format(100 * correct_val/num_val)))



Using device: cuda
1/100 train loss:0.5191 train acc:47.74% val loss:0.3703 val acc:62.97%
2/100 train loss:0.3801 train acc:61.75% val loss:0.1942 val acc:80.58%
3/100 train loss:0.3148 train acc:68.28% val loss:0.1948 val acc:80.52%
4/100 train loss:0.2495 train acc:74.81% val loss:0.2460 val acc:75.40%
5/100 train loss:0.1953 train acc:80.24% val loss:0.2543 val acc:74.57%
6/100 train loss:0.1549 train acc:84.27% val loss:0.1505 val acc:84.95%
7/100 train loss:0.1448 train acc:85.28% val loss:0.2035 val acc:79.65%
8/100 train loss:0.1432 train acc:85.45% val loss:0.1455 val acc:85.45%
9/100 train loss:0.1411 train acc:85.65% val loss:0.2312 val acc:76.88%
10/100 train loss:0.1365 train acc:86.11% val loss:0.2112 val acc:78.88%
11/100 train loss:0.1275 train acc:87.01% val loss:0.2316 val acc:76.84%
12/100 train loss:0.1295 train acc:86.81% val loss:0.1937 val acc:80.63%
13/100 train loss:0.1273 train acc:87.04% val loss:0.2103 val acc:78.97%
14/100 train loss:0.1235 train acc:87.41%

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f62eed2f680>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f62eed2f680>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/pytho

75/100 train loss:0.0441 train acc:95.35% val loss:0.1960 val acc:80.40%


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f62eed2f680>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process


76/100 train loss:0.0438 train acc:95.39% val loss:0.1575 val acc:84.25%


...

98/100 train loss:0.0383 train acc:95.94% val loss:0.1682 val acc:83.18%


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f62eed2f680>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f62eed2f680>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/pytho

99/100 train loss:0.0394 train acc:95.82% val loss:0.1980 val acc:80.20%


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f62eed2f680>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f62eed2f680>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/pytho

100/100 train loss:0.0415 train acc:95.62% val loss:0.1757 val acc:82.43%


In [ ]:
from models import mbv2_ca

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

my_save_path = '/content/drive/MyDrive/COSRMAL_CHALLENGE/task5/mobile_ca_pretrain'
os.makedirs(my_save_path,exist_ok=True)

bs = 30
train_split = 0.8
lr = 5e-4
epochs = 100
train_set = MiniDataset('/content/train/',
                        '/content/train/labels', 
                        '/content/train/crops_depth',
                        '/content/train/crops_rgb',
                        aug=True,
                        label_name=['width at the top', 'width at the bottom', 'height']
                        )
val_set = MiniDataset('/content/test/',
                      '/content/test/labels', 
                      '/content/test/crops_depth',
                      '/content/test/crops_rgb',
                      label_name=['width at the top', 'width at the bottom', 'height']
                      )
model = mbv2_ca(in_c=3, num_classes=3).to(device)

weights = torch.load('mbv2_ca.pth')
model.load_state_dict(weights, strict=False)
pretrained_weights = model.features[0][0].weight
model.features[0][0] = nn.Conv2d(4, 32, kernel_size=3, stride=2, padding=1, bias=False)
model.features[0][0].weight.data.normal_(0, 0.001)
model.features[0][0].weight.data[:, :3, :, :] = pretrained_weights

model.to(device)
optimizer = optim.Adam(model.parameters(), lr=lr,  weight_decay=1e-5)

best_loss = float('inf')
best_acc = 0

num_train = len(train_set)
num_val = len(val_set)





train_loader   = DataLoader(train_set,
                            batch_size=bs,
                            shuffle=True,
                            num_workers=1,
                            drop_last=True)
val_loader   = DataLoader(val_set,
                          batch_size=bs,
                          shuffle=True,
                          num_workers=1)

for epoch in range(epochs):
  #start_time = time.time()
  loss_train, correct_train = train_image(model, train_loader, optimizer, device, criterion = myLoss)
  loss_val, correct_val = evaluate_image(model, val_loader, device, myLoss)
  #elapsed_time = time.time() - start_time
  print("{}/{} train loss:{:.4f} train acc:{:.2f}% val loss:{:.4f} val acc:{:.2f}%".format(
      epoch+1,epochs, 1/3 * loss_train/num_train, 1/3 * 100 * correct_train/num_train,
      1/3 * loss_val/num_val, 1/3 * 100 * correct_val/num_val))
  

  # if loss_val < best_loss:
  #   best_loss = loss_val
  #   torch.save(model, os.path.join(base_path, 'audios', "bl-efficient-xl.pth"))
  
  if correct_val > best_acc:
    best_acc = correct_val
    torch.save(model.state_dict(), os.path.join(my_save_path, 
                                              "mobile-ca{:.2f}.pth".format(100 * correct_val/num_val)))



In [ ]:
from models import mbv2_ca

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

my_save_path = '/content/drive/MyDrive/COSRMAL_CHALLENGE/task4/mobile_ca_pretrain'
os.makedirs(my_save_path,exist_ok=True)

bs = 30
train_split = 0.8
lr = 1e-3
epochs = 100
train_set = MiniDataset('/content/train/',
                        '/content/train/labels', 
                        '/content/train/crops_depth',
                        '/content/train/crops_rgb',
                        aug=True,
                        label_name=['container mass']
                        )
val_set = MiniDataset('/content/test/',
                      '/content/test/labels', 
                      '/content/test/crops_depth',
                      '/content/test/crops_rgb',
                      label_name=['container mass']
                       
                      )
model = mbv2_ca(in_c=3, num_classes=1).to(device)

weights = torch.load('mbv2_ca.pth')
model.load_state_dict(weights, strict=False)
pretrained_weights = model.features[0][0].weight
model.features[0][0] = nn.Conv2d(4, 32, kernel_size=3, stride=2, padding=1, bias=False)
model.features[0][0].weight.data.normal_(0, 0.001)
model.features[0][0].weight.data[:, :3, :, :] = pretrained_weights

model.to(device)
optimizer = optim.Adam(model.parameters(), lr=lr,  weight_decay=1e-5)

best_loss = float('inf')
best_acc = 0

num_train = len(train_set)
num_val = len(val_set)





train_loader   = DataLoader(train_set,
                            batch_size=bs,
                            shuffle=True,
                            num_workers=2,
                            drop_last=True)
val_loader   = DataLoader(val_set,
                          batch_size=bs,
                          shuffle=True,
                          num_workers=2)

for epoch in range(epochs):
  #start_time = time.time()
  loss_train, correct_train = train_image(model, train_loader, optimizer, device, criterion = myLoss)
  loss_val, correct_val = evaluate_image(model, val_loader, device, myLoss)
  #elapsed_time = time.time() - start_time
  print("{}/{} train loss:{:.4f} train acc:{:.2f}% val loss:{:.4f} val acc:{:.2f}%".format(
      epoch+1,epochs, loss_train/num_train, 100 * correct_train/num_train,
      loss_val/num_val, 100 * correct_val/num_val))
  

  # if loss_val < best_loss:
  #   best_loss = loss_val
  #   torch.save(model, os.path.join(base_path, 'audios', "bl-efficient-xl.pth"))
  
  if correct_val > best_acc:
    best_acc = correct_val
    torch.save(model.state_dict(), os.path.join(my_save_path, 
                                              "mobile-ca{:.2f}.pth".format(100 * correct_val/num_val)))



## Dimension for capacity

In [6]:
class MBV2_CA_dimen(nn.Module):
    def __init__(self, dimension, in_c=4, num_classes=1, width_mult=1.):
        super(MBV2_CA_dimen, self).__init__()
        # setting of inverted residual blocks
        self.cfgs = [
            # t, c, n, s
            [1,  16, 1, 1],
            [6,  24, 2, 2],
            [6,  32, 3, 2],
            [6,  64, 4, 2],
            [6,  96, 3, 1],
            [6, 160, 3, 2],
            [6, 320, 1, 1],
        ]

        # building first layer
        input_channel = _make_divisible(32 * width_mult, 4 if width_mult == 0.1 else 8)
        layers = [conv_3x3_bn(in_c, input_channel, 2)]
        # building inverted residual blocks
        block = InvertedResidual
        for t, c, n, s in self.cfgs:
            output_channel = _make_divisible(c * width_mult, 4 if width_mult == 0.1 else 8)
            for i in range(n):
                layers.append(block(input_channel, output_channel, s if i == 0 else 1, t))
                input_channel = output_channel
        self.features = nn.Sequential(*layers)
        # building last several layers
        output_channel = _make_divisible(1280 * width_mult, 4 if width_mult == 0.1 else 8) if width_mult > 1.0 else 1280
        self.conv = conv_1x1_bn(input_channel, output_channel)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier1 = nn.Sequential(
                nn.Dropout(0.1),
                nn.Linear(output_channel+3, num_classes)
                )

        self._initialize_weights()
        
        self.dimension = dimension

    def forward(self, x):
        dimensions = self.dimension(x)
        x = self.features(x)
        
        x = self.conv(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = torch.cat((x, dimensions), dim=1)

        x = self.classifier1(x)

        return x
    
    def extract(self, x):
        x = self.features(x)
        x = self.conv(x)
        x = self.avgpool(x)
        feature = x.view(x.size(0), -1)
        x = self.classifier(feature)

        return feature, x


    def _initialize_weights(self):
        for m in self.modules():
            #print(m)
            if isinstance(m, nn.Conv2d):
                #print(m.weight.size())
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.weight.data.normal_(0, 0.01)
                m.bias.data.zero_()

In [8]:
from models import mbv2_ca

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

my_save_path = '/content/drive/MyDrive/COSRMAL_CHALLENGE/task3/dimension_trans'
os.makedirs(my_save_path,exist_ok=True)

bs = 10
train_split = 0.8
lr = 1e-3
epochs = 100
train_set = MiniDataset('/content/train/',
                        '/content/train/labels', 
                        '/content/train/crops_depth',
                        '/content/train/crops_rgb',
                        aug=True
                        )
val_set = MiniDataset('/content/test/',
                      '/content/test/labels', 
                      '/content/test/crops_depth',
                      '/content/test/crops_rgb',
                       
                      )
dimension = mbv2_ca(in_c=4, num_classes=3)
dimension.load_state_dict(torch.load('/content/drive/MyDrive/COSRMAL_CHALLENGE/task5/mobile_ca_pretrain/mobile-ca256.63.pth'))
model = MBV2_CA_dimen(dimension, in_c=4, num_classes=1)

for param in model.dimension.parameters():
    param.requires_grad = True

p_to = '/content/drive/MyDrive/COSRMAL_CHALLENGE/task3/mobile_ca_pretrain/mobile-ca82.22.pth'
model.load_state_dict(torch.load(p_to), strict=False)


model.to(device)
optimizer = optim.Adam(model.parameters(), lr=lr,  weight_decay=1e-5)

best_loss = float('inf')
best_acc = 0

num_train = len(train_set)
num_val = len(val_set)





train_loader   = DataLoader(train_set,
                            batch_size=bs,
                            shuffle=True,
                            num_workers=1,
                            drop_last=True)
val_loader   = DataLoader(val_set,
                          batch_size=bs,
                          shuffle=True,
                          num_workers=1)

for epoch in range(epochs):
  #start_time = time.time()
  loss_train, correct_train = train_image(model, train_loader, optimizer, device, criterion = myLoss)
  loss_val, correct_val = evaluate_image(model, val_loader, device, myLoss)
  #elapsed_time = time.time() - start_time
  print("{}/{} train loss:{:.4f} train acc:{:.2f}% val loss:{:.4f} val acc:{:.2f}%".format(
      epoch+1,epochs, loss_train/num_train, 100 * correct_train/num_train,
      loss_val/num_val, 100 * correct_val/num_val))
  

  # if loss_val < best_loss:
  #   best_loss = loss_val
  #   torch.save(model, os.path.join(base_path, 'audios', "bl-efficient-xl.pth"))
  
  if correct_val > best_acc:
    best_acc = correct_val
    torch.save(model.state_dict(), os.path.join(my_save_path, 
                                              "mobile-ca{:.2f}.pth".format(100 * correct_val/num_val)))



Using device: cuda
1/100 train loss:0.4332 train acc:56.43% val loss:0.5024 val acc:49.76%
2/100 train loss:0.2397 train acc:75.79% val loss:0.3770 val acc:62.30%
3/100 train loss:0.2042 train acc:79.34% val loss:0.3255 val acc:67.45%
4/100 train loss:0.1960 train acc:80.17% val loss:0.4310 val acc:56.90%
5/100 train loss:0.1841 train acc:81.35% val loss:0.4596 val acc:54.04%
6/100 train loss:0.1681 train acc:82.95% val loss:0.4172 val acc:58.28%
7/100 train loss:0.1650 train acc:83.26% val loss:0.3691 val acc:63.09%
8/100 train loss:0.1580 train acc:83.97% val loss:0.4113 val acc:58.87%
9/100 train loss:0.1497 train acc:84.79% val loss:0.4202 val acc:57.98%
10/100 train loss:0.1433 train acc:85.44% val loss:0.3437 val acc:65.63%
11/100 train loss:0.1413 train acc:85.63% val loss:0.3409 val acc:65.91%
12/100 train loss:0.1378 train acc:85.98% val loss:0.4311 val acc:56.89%
13/100 train loss:0.1327 train acc:86.49% val loss:0.4313 val acc:56.87%
14/100 train loss:0.1315 train acc:86.62%